<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN과 CNN을 사용해 시퀀스 처리하기
- 순차 데이터를 다룬다.
- 하지만 순차 데이터를 디루는 유일한 신경망은 아니다.
  - 짧은 시퀀스: Dense 네트워크가 처리 가능
  - 긴 시퀀스: Conv 네트워크가 처리 가능

## 순한 뉴런과 순환 층
<img src="https://media.vlpt.us/images/changdaeoh/post/3afed92b-978e-4173-bc56-d8453c7f2b8f/image.png"/>

- RNN은 피드포워드 신경망과 매우 비슷하지만, 
- <strong>뒷쪽</strong>으로 순환하는 연결도 있다는 점이 다르다.
- 위 처럼 <strong>각 타임  스텝 t(또는 frame)마다 x_(t)와 이전 타임 스텝의 출력인 y_(t-1)를 입력</strong>으로 받는다.
  - 첫 번째 타임 스텝은 이전 출력이 없으므로 일반적으로 0으로 설정
  - 각 뉴런은 <strong>가중치도 두 개</strong>를 가진다.(하나는 x_(t), 다른 하나는 y_(t-1)를 위한 것)
  <img src="https://media.vlpt.us/images/changdaeoh/post/355c3d96-9e65-48fd-9ad6-6b2ba9f116b3/image.png" width= 50%  height=50%/>
  - 즉, Y_(t)는 시간 t=0에서부터 모든 입력에 대한 함수가 된다.
- 동일한 뉴런을 타임 스텝마다 하나씩 표현한 위 이미지를, '시간에 따라 네트워크를 펄쳤다'고 말한다.


## 메모리 셀
- 타임 스텝 t에서 순환 뉴런의 출력은 이전 타임 스텝의 모든 입력에 대한 함수이므로, 이를 일종의 <strong>메모리</strong> 형태라고 말할 수 있다.
- <strong>메모리 셀:</strong> 타임 스텝에 걸쳐서 <strong>어떤 상태를 보존</strong>하는 신경망의 구성 요소
- 각 스텝 t에서 셀의 상태 h_(t)는 (h: hidden) 그 타임 스텝의 입력과 이전 타임 스텝의 상태에 대한 함수이다.

<br>

  <img src="https://media.vlpt.us/images/changdaeoh/post/797fafe5-5df8-424e-8adc-fe654afbb43c/image.png"/>
 

## 입력과 출력 시퀀스
- <strong>벡터-투-시퀀스</strong>
  - 각 타임 스텝에서 하나의 입력 벡터를 반복해서 주입하고, 하나의 시퀀스 출력
  - ex) 이미지 주입 -> 이미지 캡션 출력

- <strong>시퀀스-투-벡터</strong>
  - 입력 시퀀스를 주입하고 마지막을 제외한 모든 출력을 무시
  - ex) 영화 리뷰에 있는 연속된 단어를 주입 -> 감성 점수 출력(싫다(-1)~좋다(+1))

- <strong>시퀀스-투-시퀀스</strong>
  - 입력 시퀀스를 받아 출력 시퀀스를 만든다.
  - ex) 최근 N일치의 주식가격 주입 -> 각 입력값보다 하루 앞선 가격 출력
  - (즉, N-1일 전부터 내일까지)

- <strong>인코더-디코더</strong>
   - 시퀀스-투-벡터 네트워크(인코더) 뒤에 벡터-투-시퀀스 네트워크(디코더) 연결
   - ex) 한 언어의 문장을 다른 언어로 번역
   - 인코더: 해당 문장을 하나의 벡터로 변환
   - 디코더: 해당 벡터를 다른 언어의 문장으로 디코딩
   - 왜 굳이 이중  모델? 
    - 하나의 시퀀스-투-시퀀스 네트워크보다 성능이 좋다.
    - <strong>번역하기 전에 전체 문장이 주입될 때까지 기다려야 하기 때문</strong>

<br>

<img src="https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory2&fname=http%3A%2F%2Fcfile22.uf.tistory.com%2Fimage%2F99A94F465BD1990B20352F"/>

## RNN 훈련하기
- BPTT(Backpropagation Through Time)
<img src="https://media.vlpt.us/images/changdaeoh/post/333a8501-57cb-4bc6-8e11-4ca2172b4286/image.png" width=50% height=50%/>

  - 타임 스텝으로 네트워크를 펼치고(실제로 펼치진 않음)
  - 보통의 역전파를 사용하는 것
    - 정방향 패스 동안에는 <strong>모두 동일한 가중치</strong>가 적용된다(즉, 같은 매개변수 W,b사용)
    - 그래서 역전파가 진행되면 <strong>모든 타임 스텝에 걸쳐 합산</strong>될 것이다.

## 시계열 예측하기
- 시계열: 타임 스텝마다 하나 이상의 값을 가진 시퀀스
  - 단변량 시계열: 하나의 값을 가지는 시퀀스(ex) 시간당 접속 사용자 수 예측)
  - 다변량 시계열: 여러 값을 가지는 시퀀스(ex) 기업의 분기별 재정 안정성 예측 -> 수입/부채 등 고려)
  - 보통 [배치 크기, 타임 스텝 수, 차원 수] 3D배열로 나타냄(단변량은 차원 수가 1)
- 시계열 예측으로 값 대체도 가능
  - 누락된 값 예측

In [5]:
import numpy as np
def generate_time_series(batch_size, n_steps): #====batch_size만큼 n_steps 길이의 여러 시계열 생성
  freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
  time= np.linspace(0,1,n_steps) #n_steps개의 0~1사이의 값을 가지는 1d배열 생성
  series=0.5*np.sin((time-offset1)*(freq1*10+10))
  series +=0.2*np.sin((time-offset2)*(freq1*20+20))
  series +=0.1*(np.random.rand(batch_size, n_steps)-0.5) #잡음
  return series[..., np.newaxis].astype(np.float32)

In [7]:
n_steps= 50
series=generate_time_series(10000, n_steps+1)
X_train, y_train = series[:7000, :n_steps], series[:7000,-1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test=series[9000:, :n_steps], series[9000:,-1]

In [10]:
series.shape

(10000, 51, 1)

In [11]:
print(X_train.shape, y_train.shape)

(7000, 50, 1) (7000, 1)


---------------
시계열마다 1개의 값 예측

### 기준 성능
- 가장 간단한 방법: 각 시계열의 마지막 값을 그대로 예측 -> 순진한 예측이라고 함


In [17]:
import tensorflow.keras as keras
y_pred=X_valid[:,-1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.021428755

- 완전 연결 네트워크를 사용하는 방법

In [30]:
model1 = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[50,1]),
                                 keras.layers.Dense(1)
])

In [31]:
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [19]:
model1.compile(optimizer='adam', loss='mse')
model1.fit(X_train, y_train,validation_data=(X_valid, y_valid), epochs=20)

Epoch 1/20
219/219 [==============================] - 1s 2ms/step - loss: 0.2090 - val_loss: 0.0619
Epoch 2/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0517 - val_loss: 0.0343
Epoch 3/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0296 - val_loss: 0.0208
Epoch 4/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0175 - val_loss: 0.0139
Epoch 5/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0127 - val_loss: 0.0107
Epoch 6/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0098 - val_loss: 0.0090
Epoch 7/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0086 - val_loss: 0.0079
Epoch 8/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0074 - val_loss: 0.0072
Epoch 9/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0069 - val_loss: 0.0066
Epoch 10/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0061 - val_loss: 0.0060

-----------
val_loss: 0.0040까지 떨어짐

### 간단한 RNN 구현

In [27]:
model2= keras.models.Sequential(
                                   keras.layers.SimpleRNN(1, input_shape=[None, 1])) #기본적으로 tanh 활성화 함수를 사용함

In [28]:
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [29]:
model2.compile(optimizer='adam', loss='mse')
model2.fit(X_train, y_train,validation_data=(X_valid, y_valid), epochs=20)

Epoch 1/20
219/219 [==============================] - 3s 8ms/step - loss: 0.2255 - val_loss: 0.1936
Epoch 2/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1848 - val_loss: 0.1624
Epoch 3/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1564 - val_loss: 0.1382
Epoch 4/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1330 - val_loss: 0.1150
Epoch 5/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1085 - val_loss: 0.0927
Epoch 6/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0879 - val_loss: 0.0733
Epoch 7/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0675 - val_loss: 0.0569
Epoch 8/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0513 - val_loss: 0.0430
Epoch 9/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0390 - val_loss: 0.0320
Epoch 10/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0291 - val_loss: 0.0238

--------
val_loss가 0.0119로, 위 간단한 선형 모델(0.004)보다 좋지 못한 성능이지만, <br>
파라미터가 3개로, 51개인 위 모델보다 훨씬 적다.